In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

df = pd.read_csv('../data/data.csv')
df = df.dropna(subset=['state']) #罪过 最后要悄悄删掉
df = df.dropna(subset=['price'])
y = df['price']
X = df.loc[:, df.columns != 'price']
# Counting occurrences of each state
state_counts = X['state'].value_counts()

# Set common and rare states
common_states = state_counts[state_counts >= 2].index
rare_states = state_counts[state_counts < 2].index

# splitting the dataset
mask_common = X['state'].isin(common_states)
mask_rare = X['state'].isin(rare_states)

X_common, y_common = X[mask_common], y[mask_common]
X_rare, y_rare = X[mask_rare], y[mask_rare]

random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    X_common, y_common, train_size=0.98, stratify=X_common['state'], random_state=random_state
)


X_train = pd.concat([X_train, X_rare]).reset_index(drop=True)
y_train = pd.concat([y_train, y_rare]).reset_index(drop=True)


# Reset test set index
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


In [ ]:
# Preprocessing Pipeline

# preprocess with pipeline and columntransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime
import numpy as np



# collect the various features
cat_ftrs = ['status','city','state']
num_ftrs = ['bed','bath','acre_lot','house_size']
zip_ftr = 'zip_code'
broker_ftr = 'brokered_by'
street_ftr = 'street'
sold_ftr = 'prev_sold_date'


random_state = 42

class BinaryTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # NA → 0；非 NA → 1
        return np.where(pd.isna(X), 0, 1).reshape(-1,1)

class ZipCodeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.iloc[:, 0]
        zip_str = X.astype(str)
        zip_str = zip_str.str.split('.').str[0]
        # 去掉非数字字符（例如 "#####", "nan", etc.）
        zip_str = zip_str.apply(lambda s: ''.join(ch for ch in s if ch.isdigit()))
        # 空字符串 → 视为缺失
        zip_str = zip_str.replace('', np.nan)
        # 缺失 ZIP → 用 "00000" 填充
        zip_str = zip_str.fillna("00000")
        # ⭐ 最重要：统一补足为 5 位 ZIP（不足前补零）
        zip_str = zip_str.str.zfill(5)
        return zip_str.values.reshape(-1, 1)


class ZipLatLonTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, zip_lookup_path):
        self.zip_lookup_path = zip_lookup_path
    def fit(self, X, y=None):
        # 读取 ZIP → LAT, LON 映射表
        df = pd.read_csv(self.zip_lookup_path)
        # 标准化 ZIP：补齐 5 位（string）
        df['zip'] = df['zip'].astype(str).str.zfill(5)
        # 保存映射表
        self.lookup = df.set_index('zip')[['lat', 'lon']]
        return self
    def transform(self, X):
        # 如果 X 是 DataFrame，取第一列
        if isinstance(X, pd.DataFrame):
            X = X.iloc[:, 0]
        # 转字符串
        zip_str = X.astype(str).str.split('.').str[0]
        # 去掉非数字
        zip_str = zip_str.apply(lambda s: ''.join(ch for ch in s if ch.isdigit()))
        # 空字符串填 "00000"
        zip_str = zip_str.replace('', np.nan).fillna("00000")
        # 全部补齐为 5 位
        zip_str = zip_str.str.zfill(5)
        # 根据 ZIP 查 lat/lon
        lat = zip_str.map(self.lookup['lat'])
        lon = zip_str.map(self.lookup['lon'])
        # 返回两列 LAT, LON
        out = np.column_stack([lat.values, lon.values])
        return out
    
    
class PrevSoldDateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.iloc[:, 0]     # 只有一列
        else:
            X = pd.Series(X)

        dates = pd.to_datetime(X, errors='coerce')
        years = (datetime.now() - dates).dt.days / 365
        years = years.fillna(-1)

        return years.values.reshape(-1,1)



# one-hot encoder and imputer
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant',fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=True,handle_unknown='ignore'))])
# standard scaler and imputer
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=42)),
    ('scaler', StandardScaler())])

tree_numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=42))])

broker_pipe = Pipeline(steps=[
    ('binary', BinaryTransformer())
])
street_pipe = Pipeline(steps=[
    ('binary', BinaryTransformer())
])
zip_pipe = Pipeline(steps=[
    ('zip_transform', ZipCodeTransformer()),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
])

zip_latlon_pipe = Pipeline([
    ('zip_to_latlon', ZipLatLonTransformer("../data/zip_lat_lon.csv")),  
    ('imputer', SimpleImputer(strategy='median'))
])

sold_pipe = Pipeline(steps=[
    ('sold_transform', PrevSoldDateTransformer()),
    ('scaler', StandardScaler())
])
tree_sold_pipe = Pipeline(steps=[
    ('sold_transform', PrevSoldDateTransformer())
])

# collect the transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('broker', broker_pipe, broker_ftr),
        ('street', street_pipe, street_ftr),
        ('zip', zip_pipe, zip_ftr),
        ('sold', sold_pipe, sold_ftr)
    ],
    remainder='drop'
)


tree_preprocessor = ColumnTransformer(
    transformers=[
        ('num', tree_numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('broker', broker_pipe, broker_ftr),
        ('street', street_pipe, street_ftr),
        ('zip', zip_latlon_pipe, zip_ftr),
        ('sold', tree_sold_pipe, sold_ftr)
    ],
    remainder='drop'
)


xgb_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat_ftrs),
        ('broker', broker_pipe, broker_ftr),
        ('street', street_pipe, street_ftr),
        ('zip', zip_latlon_pipe, zip_ftr),
        ('sold', tree_sold_pipe, sold_ftr)
    ],
    remainder='drop'
)

X_train_tree = tree_preprocessor.fit_transform(X_train)
X_test_tree = tree_preprocessor.transform(X_test)

# 避免 log10(0) 错误，加一个非常小的常数（安全做法）
y_train_log = np.log10(y_train + 1e-6)
y_test_log = np.log10(y_test + 1e-6)

In [ ]:
# Fit RF model with best parameter

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rf_final = RandomForestRegressor(
    n_estimators=300,
    max_depth=30,
    max_features=0.3,
    min_samples_split=2,
    random_state=42,
    n_jobs=-1
)

# Fit
rf_final.fit(X_train_tree, y_train_log)


# Predict
y_pred_log = rf_final.predict(X_test_tree)


# Evaluation
rmse = mean_squared_error(y_test_log, y_pred_log) ** 0.5
r2   = r2_score(y_test_log, y_pred_log)

print("Random Forest Final Model Performance")
print(f"Test RMSE (log10): {rmse:.4f}")
print(f"Test R²   (log10): {r2:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 0. Options
# -----------------------------
sample_frac = 0.01      # 抽样比例（1%）
show_diag = True        # 是否显示 y=x 理想参考线（不想要就改 False）
seed = 42

# -----------------------------
# 1. 随机抽样
# -----------------------------
n = len(y_test_log)
rng = np.random.default_rng(seed)
m = max(1, int(n * sample_frac))
idx = rng.choice(n, size=m, replace=False)

# y_test_log 如果是 pandas Series，用 iloc；如果是 numpy array，也能兼容
y_true_sample = y_test_log.iloc[idx] if hasattr(y_test_log, "iloc") else y_test_log[idx]
y_pred_sample = y_pred_log[idx]

# -----------------------------
# 2. Plot
# -----------------------------
plt.figure(figsize=(8, 8))

plt.scatter(
    y_true_sample,
    y_pred_sample,
    alpha=0.25,
    s=10
)

# 参考线：y = x
if show_diag:
    min_val = min(np.min(y_true_sample), np.min(y_pred_sample))
    max_val = max(np.max(y_true_sample), np.max(y_pred_sample))
    plt.plot(
        [min_val, max_val],
        [min_val, max_val],
        linestyle="--",
        linewidth=1.2,
        label="Ideal: y = x"
    )

# 网格
plt.grid(True, linestyle="--", linewidth=0.6, alpha=0.6)

# 轴标签 & 标题
plt.xlabel("True log10(Price)", fontsize=12)
plt.ylabel("Predicted log10(Price)", fontsize=12)
plt.title("Random Forest: Predicted vs True (Test Set)", fontsize=14)

# 图例
plt.legend(frameon=True)

plt.tight_layout()
plt.show()


In [ ]:
# ============================
# Global Feature Importance
# Permutation Importance (Top 10)
# ============================

from sklearn.inspection import permutation_importance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 1. 取特征名（来自 tree_preprocessor）
feature_names = tree_preprocessor.get_feature_names_out()

# 2. 计算 permutation importance（在 test set 上）
perm_result = permutation_importance(
    rf_final,
    X_test_tree,
    y_test_log,
    n_repeats=2,
    random_state=42,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# 3. 整理结果
perm_df = pd.DataFrame({
    "feature": feature_names,
    "importance": perm_result.importances_mean
}).sort_values(by="importance", ascending=False)

# 4. 取 Top 10
top10_perm = perm_df.head(10)

print("Top 10 Permutation Feature Importance:")
print(top10_perm)

# 5. Plot
plt.figure(figsize=(8, 5))
plt.barh(
    top10_perm["feature"][::-1],
    top10_perm["importance"][::-1]
)
plt.xlabel("Permutation Importance (Increase in RMSE)", fontsize=12)
plt.title("Global Feature Importance (Permutation, Top 10)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# ============================
# Local Feature Importance
# SHAP Force Plots (Selected Instances Only)
# ============================

import shap

# TreeExplainer for Random Forest
explainer = shap.TreeExplainer(rf_final)

# 你指定的 test index
requested_indices = [100, 1000, 10000]
n_test = X_test_tree.shape[0]
selected_indices = [i for i in requested_indices if i < n_test]

print("Using test indices:", selected_indices)
print("Test set size:", n_test)

# 只取需要解释的样本
X_shap = X_test_tree[selected_indices]

# ⭐ 只对这几个样本计算 SHAP
shap_values = explainer.shap_values(X_shap)

# 逐个画 force plot
for i, idx in enumerate(selected_indices):
    print(f"\nSHAP force plot for test sample index = {idx}")

    shap.force_plot(
        explainer.expected_value,
        shap_values[i],
        X_shap[i],
        feature_names=feature_names,
        matplotlib=True   # 静态图，适合 PPT
    )
